## 1. load data

In [1]:
import geopandas as gpd
from os import listdir
import re

# identify shapefile in root dir
r = re.compile(".*\.shp$")
filename = next(filter(r.match, listdir()))

# load shapefile
df = gpd.read_file(filename).set_index('ID').fillna(0)
display(df.head())

,ACSTOTPOP,ACSIPOVBAS,ACSEDUCBAS,ACSTOTHH,ACSTOTHU,MINORPOP,MINORPCT,LOWINCOME,LOWINCPCT,LESSHS,...,T_PM25_B6,T_PM25_P2,T_PM25_P6,AREALAND,AREAWATER,NPL_CNT,TSDF_CNT,Shape_Leng,Shape_Area,geometry
ID,,,,,,,,,,,,,,,,,,,,,
170310101001,639,639,515,312,376,443,0.693271,247,0.386541,67,...,29%ile,87%ile,79%ile,62463.0,0.0,0,0,1689.212502,113261.122684,POLYGON ((-9759460.328199999 5164183.183399998...
170310101002,1768,1751,1116,833,955,1202,0.679864,1217,0.695031,232,...,86%ile,95%ile,92%ile,183315.0,0.0,0,0,2659.049626,332392.122294,POLYGON ((-9760181.790100001 5164417.844300002...
170310101003,1981,1881,1529,1061,1292,994,0.501767,620,0.329612,54,...,69%ile,77%ile,67%ile,133973.0,0.0,0,0,3951.080271,242919.062756,"POLYGON ((-9759674.952299999 5164425.9362, -97..."
170310102011,1417,1417,1025,619,742,916,0.646436,704,0.496824,115,...,75%ile,89%ile,88%ile,95922.0,0.0,0,0,1786.028121,173888.451215,POLYGON ((-9760948.113600001 5163250.440399997...
170310102012,4641,4628,3064,1673,1810,3615,0.778927,2569,0.555099,780,...,98%ile,94%ile,93%ile,259326.0,0.0,0,0,3623.548693,470159.705671,"POLYGON ((-9761011.5658 5163899.829499997, -97..."


In [2]:
import pandas as pd

# get field descriptions
df_dict = pd.read_excel('source/EJScreen_Index_DescriptionsV4_Pub.xlsx')

## 2. process environmental columns

See methodology here: https://www.documentcloud.org/documents/5019105-NRDC-Cumulative-Impacts-Method.html

In [3]:
env_indicators = [
    'DSLPM',
    'CANCER',
    'RESP',
    'PTRAF',
    'PWDIS',
    'PNPL',
    'PRMP',
    'PTSDF',
    'OZONE',
    'PM25',
    'PRE1960PCT']

env_dict = {
    'CANCER': 'CANCR',
    'PRE1960PCT': 'LDPNT'
}

display(df_dict[df_dict.FIELD_NAME.isin(env_indicators)])

,FIELD_NAME,DESCRIPTION,CATEGORY
19,PRE1960PCT,Pct. Housing Units Built Prior to 1960 (lead p...,Environmental Factor
26,DSLPM,NATA Diesel Particulate Matter,Environmental Factor
27,CANCER,NATA Cancer Risk,Environmental Factor
28,RESP,NATA Respiratory Hazard Index,Environmental Factor
29,PTRAF,Traffic Proximity,Environmental Factor
30,PWDIS,Wastewater Discharge Indicator,Environmental Factor
31,PNPL,Superfund Proximity,Environmental Factor
32,PRMP,RMP Proximity,Environmental Factor
33,PTSDF,TSDF Proximity,Environmental Factor
34,OZONE,Ozone Concentration Score,Environmental Factor


### 2a. calculate percentile

To calculate each factor's "quintile" score, we first have to calculate its raw percentile. (I don't remember what the c stands for here.)

In [4]:
from scipy import stats

c_env_df = df.filter(env_indicators).apply(lambda series: series.apply(lambda x: stats.percentileofscore(series, x, kind='strict')))
c_env_df.rename(lambda x: 'C_' + env_dict.get(x,x), axis='columns', inplace=True)

display(c_env_df.head())

,C_DSLPM,C_CANCR,C_RESP,C_PTRAF,C_PWDIS,C_PNPL,C_PRMP,C_PTSDF,C_OZONE,C_PM25,C_LDPNT
ID,,,,,,,,,,,
170310101001,63.882515,67.094998,75.493346,0.000000,0.000000,0.826067,36.714089,21.018816,89.765948,0.000000,12.666361
170310101002,63.882515,67.094998,75.493346,1.376778,58.742542,7.572281,39.513538,25.103258,89.765948,0.000000,59.385039
170310101003,63.882515,67.094998,75.493346,0.000000,0.000000,8.122992,35.979807,20.743460,89.765948,0.000000,48.003671
170310102011,73.428178,50.114732,64.203763,10.096374,91.142726,6.470858,37.769619,42.129417,82.744378,0.550711,22.900413
170310102012,73.428178,50.114732,64.203763,9.729234,90.729693,6.149610,42.634236,36.714089,82.744378,0.550711,64.066085


### 2b. bin percentiles into quintiles

The percentiles are then reduced to a simple score from 1 to 5. (The s here stands for "score".)

In [5]:
bins = [-1,20,40,60,80,100]
labels = [1,2,3,4,5]

s_env_df = c_env_df.apply(lambda series: pd.cut(series, bins, labels=labels).values.add_categories(0).astype(int))
s_env_df.rename(lambda x: x.replace('C_', 'S_'), axis='columns', inplace=True)

display(s_env_df.head())

,S_DSLPM,S_CANCR,S_RESP,S_PTRAF,S_PWDIS,S_PNPL,S_PRMP,S_PTSDF,S_OZONE,S_PM25,S_LDPNT
ID,,,,,,,,,,,
170310101001,4,4,4,1,1,1,2,2,5,1,1
170310101002,4,4,4,1,3,1,2,2,5,1,3
170310101003,4,4,4,1,1,1,2,2,5,1,3
170310102011,4,3,4,1,5,1,2,3,5,1,2
170310102012,4,3,4,1,5,1,3,2,5,1,4


### 2c. sum up quintiles... and bin those too

We add up the factors (s_env_total), calculate a percentile from that (c_env_total) and bin *that.*

In [6]:
s_env_df['S_ENV_TOTAL'] = s_env_df.sum(axis=1)

series = s_env_df['S_ENV_TOTAL']
s_env_df['C_ENV_TOTAL'] = series.apply(lambda x: stats.percentileofscore(series, x, kind='strict'))
s_env_df['S_ENV_OVERALL'] = pd.cut(s_env_df['C_ENV_TOTAL'], bins, labels=labels).astype(int)

display(s_env_df.head())

,S_DSLPM,S_CANCR,S_RESP,S_PTRAF,S_PWDIS,S_PNPL,S_PRMP,S_PTSDF,S_OZONE,S_PM25,S_LDPNT,S_ENV_TOTAL,C_ENV_TOTAL,S_ENV_OVERALL
ID,,,,,,,,,,,,,,
170310101001,4,4,4,1,1,1,2,2,5,1,1,26,7.067462,1
170310101002,4,4,4,1,3,1,2,2,5,1,3,30,29.463056,2
170310101003,4,4,4,1,1,1,2,2,5,1,3,28,16.613125,1
170310102011,4,3,4,1,5,1,2,3,5,1,2,31,37.356586,2
170310102012,4,3,4,1,5,1,3,2,5,1,4,33,51.124369,3


## 3. process population columns

See methodology here: https://www.documentcloud.org/documents/5019105-NRDC-Cumulative-Impacts-Method.html

In [7]:
pop_indicators = [
    'LOWINCPCT',
    'MINORPCT',
    'LESSHSPCT',
    'LINGISOPCT',
    'UNDER5PCT',
    'OVER64PCT'
]

pop_dict = {
    'LOWINCPCT': 'LWINCPCT',
    'MINORPCT': 'MINORPCT',
    'LESSHSPCT': 'LESHSPCT',
    'LINGISOPCT': 'LNGISPCT',
    'UNDER5PCT': 'UNDR5PCT',
    'OVER64PCT': 'OVR64PCT'
}

display(df_dict[df_dict.FIELD_NAME.isin(pop_indicators)])

,FIELD_NAME,DESCRIPTION,CATEGORY
7,MINORPCT,Pct. Minority Population,Population
9,LOWINCPCT,Pct. Low Income (<2x poverty level),Income/Poverty
11,LESSHSPCT,Pct. Less than High School Education,Education
13,LINGISOPCT,Pct. Linguistically Isolated,Language
15,UNDER5PCT,Pct. Under Age 5,Population
17,OVER64PCT,Pct. Over Age 64,Population


### 3a. calculate percentile

To calculate each factor's "quintile" score, we first have to calculate its raw percentile. (I don't remember what the c stands for here.)

In [8]:
from scipy import stats

c_pop_df = df.filter(pop_indicators).apply(lambda series: series.apply(lambda x: stats.percentileofscore(series, x, kind='strict')))
c_pop_df.rename(lambda x: 'C_' + pop_dict.get(x,x), axis='columns', inplace=True)

display(c_pop_df.head())

,C_LWINCPCT,C_MINORPCT,C_LESHSPCT,C_LNGISPCT,C_UNDR5PCT,C_OVR64PCT
ID,,,,,,
170310101001,44.561726,42.726021,45.387793,0.000000,0.000000,4.726939
170310101002,86.691143,42.175310,65.213401,49.793483,82.193667,13.630106
170310101003,36.897659,32.491969,18.127581,0.000000,25.516292,14.089032
170310102011,59.063791,40.385498,39.743001,71.913722,39.238183,44.561726
170310102012,67.691602,48.324920,74.713171,72.372648,73.474071,12.023864


### 3b. bin percentiles into quintiles

The percentiles are then reduced to a simple score from 1 to 5. (The s here stands for "score".)

In [9]:
bins = [-1,20,40,60,80,100]
labels = [1,2,3,4,5]

s_pop_df = c_pop_df.apply(lambda series: pd.cut(series, bins, labels=labels).values.add_categories(0).astype(int))
s_pop_df.rename(lambda x: x.replace('C_', 'S_'), axis='columns', inplace=True)

display(s_pop_df.head())

,S_LWINCPCT,S_MINORPCT,S_LESHSPCT,S_LNGISPCT,S_UNDR5PCT,S_OVR64PCT
ID,,,,,,
170310101001,3,3,3,1,1,1
170310101002,5,3,4,3,5,1
170310101003,2,2,1,1,2,1
170310102011,3,3,2,4,2,3
170310102012,4,3,4,4,4,1


### 3c. sum up quintiles... and bin those too

We add up the factors (s_pop_total), calculate a percentile from that (c_pop_total) and bin *that.*

In [10]:
s_pop_df['S_POP_TOTAL'] = s_pop_df.sum(axis=1)

series = s_pop_df['S_POP_TOTAL']
s_pop_df['C_POP_TOTAL'] = series.apply(lambda x: stats.percentileofscore(series, x, kind='strict'))
s_pop_df['S_POP_OVERALL'] = pd.cut(s_pop_df['C_POP_TOTAL'], bins, labels=labels).astype(int)

display(s_pop_df.head())

,S_LWINCPCT,S_MINORPCT,S_LESHSPCT,S_LNGISPCT,S_UNDR5PCT,S_OVR64PCT,S_POP_TOTAL,C_POP_TOTAL,S_POP_OVERALL
ID,,,,,,,,,
170310101001,3,3,3,1,1,1,12,12.069757,1
170310101002,5,3,4,3,5,1,21,67.416246,4
170310101003,2,2,1,1,2,1,9,2.707664,1
170310102011,3,3,2,4,2,3,17,36.714089,2
170310102012,4,3,4,4,4,1,20,57.870583,3


## 4. merge everything

Stick everything in a single dataframe for export, plus add an overall score column.

In [11]:
merged_df = pd.concat([df, c_pop_df, c_env_df, s_pop_df, s_env_df], axis=1, sort=True)
merged_df['S_OVERALL'] = merged_df['S_POP_OVERALL'] + merged_df['S_ENV_OVERALL']

display(merged_df.head())

,ACSTOTPOP,ACSIPOVBAS,ACSEDUCBAS,ACSTOTHH,ACSTOTHU,MINORPOP,MINORPCT,LOWINCOME,LOWINCPCT,LESSHS,...,S_PNPL,S_PRMP,S_PTSDF,S_OZONE,S_PM25,S_LDPNT,S_ENV_TOTAL,C_ENV_TOTAL,S_ENV_OVERALL,S_OVERALL
ID,,,,,,,,,,,,,,,,,,,,,
170310101001,639,639,515,312,376,443,0.693271,247,0.386541,67,...,1,2,2,5,1,1,26,7.067462,1,2
170310101002,1768,1751,1116,833,955,1202,0.679864,1217,0.695031,232,...,1,2,2,5,1,3,30,29.463056,2,6
170310101003,1981,1881,1529,1061,1292,994,0.501767,620,0.329612,54,...,1,2,2,5,1,3,28,16.613125,1,2
170310102011,1417,1417,1025,619,742,916,0.646436,704,0.496824,115,...,1,2,3,5,1,2,31,37.356586,2,4
170310102012,4641,4628,3064,1673,1810,3615,0.778927,2569,0.555099,780,...,1,3,2,5,1,4,33,51.124369,3,6


## 5. export

In [12]:
import os
import shutil

directory = filename.split('.')[0] + '__scored'

if not os.path.exists(directory):
    os.makedirs(directory)

out_df = gpd.GeoDataFrame(merged_df).reset_index().round(3)
out_df.columns = out_df.columns.str.lower()
out_df.crs = df.crs

# make shapefile
out_df.to_file(directory)
shutil.make_archive(directory, 'zip', directory)

# make csv
if not os.path.exists('output'):
    os.makedirs('output')

pd.DataFrame(out_df).drop('geometry', axis=1).to_csv('output/' + directory + '.csv')